In [1]:
from semanticscholar import SemanticScholar
from bs4 import BeautifulSoup
from selenium import webdriver
import requests

import random
import time
import csv
import regex as re

from pandas import DataFrame, read_csv, option_context

In [59]:
def print_all(text):
    with option_context('display.max_rows', None, 'display.max_columns', None):  
        # more options can be specified also
        print(text)

In [92]:
def write_csv(data, filename):
    with open(filename, 'w', encoding='UTF8') as f:
        # create the csv writer
        writer = csv.writer(f)
        # write a row to the csv file
        writer.writerow(data)

## First experiment

In [2]:
sch = SemanticScholar()
results = sch.search_paper('+"gender bias"', fields=['title','year', 'authors', 'publicationTypes', 'citationCount', 'url', 'fieldsOfStudy', 's2FieldsOfStudy', 'journal'])
print(f'{results.total} results.', f'First occurrence: {results[0].title}.')

2366962 results. First occurrence: Gender Bias in Machine Translation.


In [49]:
data = []
for result in results[0:10]:
    for author in result['authors']:
        data.append({'Title': result['title'], 
                    'Year': result['year'],
                    'Author': author['name'],
                    'PublicationType': result['publicationTypes'], 
                    'CitationCount': result['citationCount'],
                    'FieldsOfStudy': result['fieldsOfStudy'],
                    'FieldsOfStudy_s2': result['s2FieldsOfStudy'],
                    'Journal': result['journal'],
                    'URL': result['url']})

In [70]:
results[0]

{'paperId': '551f36edf960d196f8499bf5db37537dd6433318', 'url': 'https://www.semanticscholar.org/paper/551f36edf960d196f8499bf5db37537dd6433318', 'title': 'Stag Parties Linger: Continued Gender Bias in a Female-Rich Scientific Discipline', 'year': 2012, 'citationCount': 95, 'fieldsOfStudy': ['Medicine'], 's2FieldsOfStudy': [{'category': 'Medicine', 'source': 'external'}, {'category': 'Psychology', 'source': 's2-fos-model'}], 'publicationTypes': ['JournalArticle'], 'journal': {'name': 'PLoS ONE', 'volume': '7'}, 'authors': [{'authorId': '3900551', 'name': 'L. Isbell'}, {'authorId': '6896193', 'name': 'T. Young'}, {'authorId': '7032312', 'name': 'A. Harcourt'}]}

In [41]:
type(data[0]['Journal'])

dict

In [5]:
len(results)

100

In [61]:
df = DataFrame(data, columns = ['Title', 'Year', 'Author', 'PublicationType', 'CitationCount', 'URL', 'FieldsOfStudy', 'FieldsOfStudy_s2', 'Journal'])
df[['FieldsOfStudy', 'Journal']].head(10)

,FieldsOfStudy,Journal
0,[Computer Science],{'name': 'Transactions of the Association for ...
1,[Computer Science],{'name': 'Transactions of the Association for ...
2,[Computer Science],{'name': 'Transactions of the Association for ...
3,[Computer Science],{'name': 'Transactions of the Association for ...
4,[Computer Science],{'name': 'Transactions of the Association for ...
5,[Computer Science],{'pages': '15-20'}
6,[Computer Science],{'pages': '15-20'}
7,[Computer Science],{'pages': '15-20'}
8,[Computer Science],{'pages': '15-20'}
9,[Computer Science],{'pages': '15-20'}


In [64]:
df.Journal.iloc[0].keys()

dict_keys(['name', 'pages', 'volume'])

In [ ]:
df['JournalTitle'] = df['Journal'].apply(lambda x: x.get('name') if 'name' in x.keys() else None)
df['JournalTitle']

In [ ]:
l = ['name']
journal = [x.get('name') if 'name' in x.keys() else 'F' for x in df['Journal'] ] #.apply(lambda x: x['name'])
journal

## Data 2012-2023

In [ ]:
data_authors = []
df_authors = DataFrame(columns = ['Title', 
    'Year', 
    'Author', 
    'PublicationType', 
    'CitationCount', 
    'FieldsOfStudy', 
    'FieldsOfStudy_s2', 
    'Journal',
    'URL'])

sch = SemanticScholar()
for year in range(2012, 2024):
    results = sch.search_paper('+"gender bias"', 
        fields=['title','year', 'authors', 'publicationTypes', 'citationCount', 'fieldsOfStudy', 's2FieldsOfStudy', 'journal', 'url'],
        year=year)
    print(len(results))
    for result in results:
        for author in result['authors']:
            data_authors.append({'Title': result['title'], 
                    'Year': result['year'],
                    'Author': author['name'],
                    'PublicationType': result['publicationTypes'], 
                    'CitationCount': result['citationCount'],
                    'FieldsOfStudy': result['fieldsOfStudy'],
                    'FieldsOfStudy_s2': result['s2FieldsOfStudy'],
                    'Journal': result['journal'],
                    'URL': result['url']})
    print(f'Since 2012 until {year} there are {len(data_authors)} results')
    

In [2]:
data_journals= []
df_journals = DataFrame(columns = ['Title', 
    'Year', 
    'FieldsOfStudy', 
    'FieldsOfStudy_s2', 
    'Journal'])

sch = SemanticScholar()
for year in range(2012, 2024):
    results = sch.search_paper('+"gender bias"', 
        fields=['title','year', 'fieldsOfStudy', 's2FieldsOfStudy', 'journal'],
        year=year)
    print(len(results))
    for result in results:
        data_journals.append({'Title': result['title'], 
            'Year': result['year'],
            'FieldsOfStudy': result['fieldsOfStudy'],
            'FieldsOfStudy_s2': result['s2FieldsOfStudy'],
            'Journal': result['journal']})
    print(f'Since 2012 until {year} there are {len(data_journals)} results')
    

100
Since 2012 until 2012 there are 9999 results
100
Since 2012 until 2013 there are 19998 results
100
Since 2012 until 2014 there are 27297 results
100
Since 2012 until 2015 there are 37296 results
100
Since 2012 until 2016 there are 47295 results
100
Since 2012 until 2017 there are 57294 results
100
Since 2012 until 2018 there are 67293 results
100
Since 2012 until 2019 there are 77292 results
100
Since 2012 until 2020 there are 87291 results
100
Since 2012 until 2021 there are 97290 results
100
Since 2012 until 2022 there are 107289 results
100
Since 2012 until 2023 there are 117288 results


In [3]:
dict_TitleJournal = {}
for data in data_journals:
    #print(data['Journal'].keys())
    try:
        dict_TitleJournal[data['Title']] = [data['Year'], data['Journal']['name'], data['FieldsOfStudy'], data['FieldsOfStudy_s2']]
    except:
        pass

In [35]:
dict_TitleField = {}
for data in data_journals:
    #print(data['Journal'].keys())
    try:
        dict_TitleField[data['Title']] = data['FieldsOfStudy']
    except:
        pass

In [4]:
print(data_journals[13])

{'Title': "Mechanisms of women's own-gender bias and sex differences in memory for faces", 'Year': 2012, 'FieldsOfStudy': ['Psychology'], 'FieldsOfStudy_s2': [{'category': 'Psychology', 'source': 'external'}, {'category': 'Psychology', 'source': 's2-fos-model'}], 'Journal': {'name': '', 'volume': ''}}


In [5]:
print(len(dict_TitleJournal))
print(dict_TitleJournal["Mechanisms of women's own-gender bias and sex differences in memory for faces"])

102391
[2012, '', ['Psychology'], [{'category': 'Psychology', 'source': 'external'}, {'category': 'Psychology', 'source': 's2-fos-model'}]]


In [34]:
i = 0

with open('dict_TitleField.csv', 'w') as f:
    for key in dict_TitleJournal.keys():
        try:
            f.write("%s,%s\n"%(key,dict_TitleField[key]))
        except:
            i += 1

print(i)


3359


write to csv

In [18]:
# open the file in the write mode
with open('data/semanticScholar_try_+gender_bias.csv', 'w', encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)
    # write a row to the csv file
    writer.writerow(data_authors)

write to a DataFrame

In [5]:
df_authors = DataFrame(data_authors, columns = ['Title', 'Year', 'Author', 'PublicationType', 'CitationCount', 'URL'])
print(df_authors['Author'].nunique())

247671


check duplicates

In [88]:
df_au_dd = df_authors.drop_duplicates(subset=["Title", "Author"], keep="first")
print(len(df_authors), len(df_au_dd))

396553 357789


drop duplicates inplace

In [130]:
df_authors.drop_duplicates(subset=["Title", "Author"], keep="first", inplace=True, ignore_index=True)

cast Year as int

In [92]:
df_authors.Year.astype('Int64')

0         2012
1         2012
2         2012
3         2012
4         2012
          ... 
396548    2023
396549    2023
396550    2023
396551    2023
396552    2023
Name: Year, Length: 357789, dtype: Int64

Show main info about the Dataframe and number of unique names

In [93]:
print(df_authors.info())
print(df_authors.isnull().sum())
df_authors.Author.nunique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 357789 entries, 0 to 396552
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Title            357789 non-null  object
 1   Year             357789 non-null  int64 
 2   Author           357789 non-null  object
 3   PublicationType  229206 non-null  object
 4   CitationCount    357789 non-null  int64 
 5   URL              357789 non-null  object
dtypes: int64(2), object(4)
memory usage: 19.1+ MB
None
Title                   0
Year                    0
Author                  0
PublicationType    128583
CitationCount           0
URL                     0
dtype: int64


247671

In [38]:
df_authors.to_csv('data/authors_2012-2023.csv')

### Sharpen the results

In [4]:
df_authors = read_csv('data/authors_2012-2023.csv')

In [9]:
print('3 patterns', len(df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[Bb]ias|[Gg]ender(.*)[Gg]ap|[Gg]ender(.*)[iI]nequalit')]))
print('Gender bias', len(df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[Bb]ias')]))
print('Gender Gap', len(df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[Gg]ap')]))
print('Gender Inequality', len(df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[iI]nequalit')]))
print('Gender difference', len(df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[Dd]ifference')]))

C:\Users\afedorova\AppData\Local\Temp\ipykernel_12032\1389154377.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  print('3 patterns', len(df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[Bb]ias|[Gg]ender(.*)[Gg]ap|[Gg]ender(.*)[iI]nequalit')]))


3 patterns 16353


C:\Users\afedorova\AppData\Local\Temp\ipykernel_12032\1389154377.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  print('Gender bias', len(df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[Bb]ias')]))


Gender bias 8324


C:\Users\afedorova\AppData\Local\Temp\ipykernel_12032\1389154377.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  print('Gender Gap', len(df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[Gg]ap')]))


Gender Gap 5327


C:\Users\afedorova\AppData\Local\Temp\ipykernel_12032\1389154377.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  print('Gender Inequality', len(df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[iI]nequalit')]))


Gender Inequality 2881


C:\Users\afedorova\AppData\Local\Temp\ipykernel_12032\1389154377.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  print('Gender difference', len(df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[Dd]ifference')]))


Gender difference 30100


In [377]:
df_authors_re = df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[Bb]ias|[Gg]ender(.*)[Gg]ap|[Gg]ender(.*)[dD]ifference')]
df_authors_re = df_authors_re.reset_index(drop=True)
#df_authors_re.to_csv('data/authors_2012-2023_regex.csv')

C:\Users\afedorova\AppData\Local\Temp\ipykernel_1824\2789606413.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_authors_re = df_authors[df_authors.Title.str.contains('[Gg]ender(.*)[Bb]ias|[Gg]ender(.*)[Gg]ap|[Gg]ender(.*)[dD]ifference')]


In [378]:
print('number of row in the initial DF: ', len(df_authors), '\n', 'number of row in the DF after regex: ', len(df_authors_re), sep='')
print('number of unique articles before search precision:', df_authors['Title'].nunique())
print('number of unique article titles after search precision:',df_authors_re['Title'].nunique()) #.to_csv('data/regex_search.csv')
print('number of unique author names after search precision:',df_authors_re['Author'].nunique()) #.to_csv('data/regex_search.csv')

number of row in the initial DF: 357789
number of row in the DF after regex: 39287
number of unique articles before search precision: 99189
number of unique article titles after search precision: 10551
number of unique author names after search precision: 33793


### Data cleaning

#### Clean chinese characters

Search for any Chinese characters in Author names, as this is the crucial informaiton for our data.

In [320]:
df_authors_re[['Title', 'Author']][df_authors_re['Author'].str.contains(r'[\u4e00-\u9FFF]', regex=True)]

,Title,Author
4089,Gender Roles: Biases or Differences?,魏静
4179,從本土企業董事性別比例平衡與績效的關係看性別平等意涵; Corporate Board Ge...,王孟洵
7925,Gender Bias and Resistance in Edna O'Brien's C...,張崇旂
7996,Gender Bias Against Foreign Wives : Analysis o...,加藤 直子
9900,Lower Melatonin Secretion in Older Females: Ge...,Kenji 賢史 Obayashi 大林
9901,Lower Melatonin Secretion in Older Females: Ge...,Keigo 圭吾 Saeki 佐伯
9902,Lower Melatonin Secretion in Older Females: Ge...,Nobuhiro 庸浩 Tone 刀根
9903,Lower Melatonin Secretion in Older Females: Ge...,Junko 淳子 Iwamoto 岩本
9904,Lower Melatonin Secretion in Older Females: Ge...,Kimie 季美恵 Miyata 宮田
9905,Lower Melatonin Secretion in Older Females: Ge...,Yoshito 義人 Ikada 筏


Often if the author name has Chinese characters, also title would have them, but they are always accompanied with an English title, so we can remove Chinese characters and be sure, that the title will not ne NaN.

In [379]:
df_authors_re['Title'] = df_authors_re['Title'].str.replace(r'[\u4e00-\u9FFF]','', regex=True)

We will partially drop rows, where the author column only contains Chinese characters, if in this case the preliminary analysis has shown, that's it'snot possible to find out the full name of the author. For some titles this information was found and will be udated.

On the other hand we will keep the rows with mixed Latin and Chinese characters, as they provide the name information and just remove Chinese characters.

In [380]:
titles_drop = ['Deuce or advantage? : examining gender bias in online coverage of professional tennis through a mixed methodology',
    'Gender Roles: Biases or Differences?',
    'Gender Differences in Conversation',
    'Analysis of Gender Differences in Color Reaction in TV Show You Are the One',
    'Gender Bias and Stereotypes in English Example Sentences—Taking Linguistic Academic Journals as Example']
title_name_drop = ['; Corporate Board Gender Diversity and Firm Financial Performance：The Institutional Investor Bias in Taiwan',
    '王孟洵']

print(len(df_authors_re))

i_toDrop = df_authors_re[(df_authors_re.Title.isin(titles_drop)) |\
    ((df_authors_re.Title == title_name_drop[0]) & \
    (df_authors_re.Author == title_name_drop[1]))].index
df_authors_re = df_authors_re.drop(i_toDrop)

print(len(df_authors_re))

39287
39280


In [381]:
df_authors_re['Author'][df_authors_re['Author'].str.contains(r'[\u4e00-\u9FFF]', regex=True) & df_authors_re['Author'].str.contains(r'[a-zA-Z]', regex=True)] \
    = df_authors_re['Author'].str.replace(r'[\u4e00-\u9FFF]','', regex=True)

C:\Users\afedorova\AppData\Local\Temp\ipykernel_1824\4128585737.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_authors_re['Author'][df_authors_re['Author'].str.contains(r'[\u4e00-\u9FFF]', regex=True) & df_authors_re['Author'].str.contains(r'[a-zA-Z]', regex=True)] \


In [382]:
replace_ChinCharNames = {'張崇旂': 'Tsung-Chi Chang', #'Gender bias and resistance in Edna O’Brien’s “country girls' trilogy”' 
    '加藤 直子': 'Naoko Kato', #'Gender Bias Against Foreign Wives : Analysis of International Divorce Cases in Japan' 
    '三好 葉月': 'Hazuki Moyoshi', #'Reducing unconscious gender bias through workshop with co-speculative design'
    '加藤 朗': 'Akira Kato'} #'Reducing unconscious gender bias through workshop with co-speculative design'

df_authors_re['Author'] = df_authors_re['Author'].replace(replace_ChinCharNames, regex=True)

In [383]:
df_authors_re[['Title','Author']][df_authors_re['Author'].str.contains(r'[\u4e00-\u9FFF]', regex=True)]

,Title,Author


#### Search for Hangul (Korean characters)

In [327]:
df_authors_re[['Title','Author']][df_authors_re['Author'].str.contains(r'[\u3131-\u3163\uac00-\ud7a3]', regex=True)]

,Title,Author
154,Gender Bias in Schooling,김기승
3518,한국가족 내 부부간 성관계에 투영된 젠더 격차(Gender Gap),함인희
7894,Gender Differences in Oral Health Literacy rel...,이규은
7895,Gender Differences in Oral Health Literacy rel...,염영희
7896,Gender Differences in Oral Health Literacy rel...,김상숙
7897,Gender Differences in Oral Health Literacy rel...,한정희
7995,Cultural Linguistic Study on the Chinese Gende...,김현태
8730,A Study on Factors Influencing Elders' Suicida...,구춘영
8731,A Study on Factors Influencing Elders' Suicida...,김정순
8732,A Study on Factors Influencing Elders' Suicida...,유정옥


In [384]:
replace_KorCharNames = {'김기승': 'Seung-Gi Kim',
    'M. Milligan': 'Michael A. Milligan',
    '이규은': 'Kyu Eun lee', #Gender Differences in Oral Health Literacy
    '염영희': 'Young-Hee Yom', #Gender Differences in Oral Health Literacy
    '김상숙': 'Sang Suk Kim', #Gender Differences in Oral Health Literacy
    '한정희': 'Jung Hee Han', #Gender Differences in Oral Health Literacy
    '김현태': 'Kim Hyun Tae', #'Korean' 'Male' #'Cultural Linguistic Study on the Chinese Gender-biased Terms against Women'
    '구춘영': 'Chun Young Koo', #A Study on Factors Influencing Elders' Suicidal Ideation: Focused on Comparison of Gender Differences
    '김정순': 'Jung Soon Kim', #A Study on Factors Influencing Elders' Suicidal Ideation: Focused on Comparison of Gender Differences
    '유정옥': 'Jungok Yu', #A Study on Factors Influencing Elders' Suicidal Ideation: Focused on Comparison of Gender Differences
    '김세현': 'Sehyoun Kim'} #An Audience-response Questionnaire Survey on Gender-biased Language in Korean Subtitling}

df_authors_re['Author'] = df_authors_re['Author'].replace(replace_KorCharNames, regex=True)
i_toDrop = df_authors_re[df_authors_re.Title == '한국가족 내 부부간 성관계에 투영된 젠더 격차(Gender Gap)'].index
df_authors_re = df_authors_re.drop(i_toDrop)

len(df_authors_re)


39279

#### Check rows with names, that only contain one word

In [392]:
df_authors_re[['Title', 'Author']][~df_authors_re.Author.str.contains('\s')] #.to_csv('data/onewordnames.csv')
    #'^[\S]\.[\S].')] 
    # #& ~df_authors_re.Author.str.contains('^[a-zA-Z]\.\s[\S].*')]


,Title,Author


In [370]:
print(df_authors_re['Author'][df_authors_re.Author.isin(names_short)])

Series([], Name: Author, dtype: object)


In [393]:
names_update = {'K: Hughes': 'Karen D. Hughes',
'S. Albanesi': 'Stefania Albanesi',
'Nyfed': 'Aysegül Sahin',
'Bankole': 'Bankole Fasanya',
'Maranda': 'Maranda Mcbride',
'Pope-Ford': 'Regina Pope-Ford',
'Ntuen': 'Celestine Ntuen',
'E. Alpha': 'Ayande Alpha',
'Quinetta': 'Quinetta M. Roberson',
'Hrebickova': 'Hana Hrebickova',
#'M.': 'Gregory S. Parks',
'E. Oyeniyi': 'Akande Samson Oyeniyi',
'ZhangXiaofei': 'Zhang Xiaofei',
'GuoXitong': 'Guo Xitong',
'LaiKee-hung': 'Lai Kee-hung',
'GuoFeng': 'Guo Feng',
'LiChenlei': 'Li Chenlei',
'CohenAaron': 'Aaron Cohen',
'Ein-MorEliana': 'Eliana Einmor',
'StessmanJochanan': 'Jochanan Stessman',
'B. Rustin': 'Brett Rustin Nielsen',
'R. Foels': 'Rob Foels',
'I. GuerdjikovaAnna': 'Anna I. Guerdjikova',
'J. BlomThomas': 'Thomas J. Blom',
'MoriNicole': 'Nicole Mori',
'CasutoLeah': 'Leah Casuto',
'KeckPaul E.Jr.': 'Paul E.Jr. Keck',
'L. McElroySusan': 'Susan McElroy',
'TerranovaVictoria': 'Victoria Terranova',
'M. VandiverDonna': 'Donna Vandiver',
'GuoXitong': 'Guo Xitong',
'HanXiaocui': 'Han Xiaocui',
'ZhangXiaofei': 'Zhang Xiaofei',
'DangYuanyuan': 'Dang Yuanyuan',
'ChenChun': 'Chen Chun',
'Corrine': 'Corinne Wilsey',
'Hazel-FernandezLeslie': 'Leslie Hazel-Fernandez',
'NeroDamion': 'Damion Nero',
'MoretzChad': 'Chad Moretz',
'SlabaughLane': 'Lane Slabaugh',
'MeahYunus': 'Yunus Meah',
'BaltzJean': 'Jean Baltz',
'CostantinoMary': 'Mary Costantino',
'BouchardJonathan': 'Jonathan Bouchard',
'Omiya': 'Kazuto Omiya',
'S. Gilli': 'Sofia Gilli',
'TempletonKim': 'Kim Templeton',
'S. Javed': 'Saira Javed',
'Umm-e-Siddiqa': 'Umme Siddiqa',
'IramZehraBokharey': 'Zehra Iram Bokharey',
'J. Turabián': 'Jose Turabián',
'ra Moreno-Ruiz': 'Sandra Moreno-Ruiz',
'R. Cucho-Jove': 'Raul Cucho-Jove',
'ro Villarin-Castro': 'Alejandro Villarin-Castro',
'P. Bhattacharya': 'Prabir Bhattacharya',
'Amsterdam': 'Marja J Aartsen',
'S. Yamtinah': 'Sri Yamtinah',
'M. Masykuri': 'Mohammad Masykuri',
'A. Ashadi': 'Ashadi Ashadi',
'A. Shidiq': 'Ari Syahidul Shidiq',
'CarapinhaRené': 'René Carapinha',
'E. Choo': 'Esther Choo',
'R. DeMayo': 'Robert DeMayo',
'Warnakula':'Upuli Warnakula',
"Stephen D. O'Connell": 'Stephen D. OConnell',
"O'Connell": 'Stephen D. OConnell',
'Baloglu': 'Mustafa Baloğlu',
'AuerMichael': 'Michael Auer',
'FrauscherBirgit': 'Birgit Frauscher',
'HochleitnerMargarethe': 'Margarethe Hochleitner',
'HöglBirgit': 'Birgit Högl',
'Mzamu-Kampanje': 'Jessica Mzamu-Kampanje',
'SudomKerry': 'Kerry Sudom',
'GuérinEva': 'Eva Guérin',
'E. C. LeeJennifer': 'Jennifer E.C. Lee',
'S. Ananiadou': 'Sophia Ananiadou',
'Odbal': 'Odbal Odbal',
'Y. Hirota': 'Yusuke Hirota',
'y-hirota': 'Yuta Nakashima', 
'O. Steinlein': 'Ortrud Steinlein',
'M. Reithmair': 'Marlene Reithmair',
'ya Syunyaeva': 'Zulfiya Syunyaeva',
'E. Sattler': 'Elke C Sattler',
'Anthonius':'Anthonius de Boer',
'Denier': 'Nicole Denier',
'Gag': 'Gagandeep Singh',
'M. Abiodun':'Abiodun M. Gesinde',
'Js Oluwafunto':'Oluwafunto J. Sanu'}

df_authors_re['Author'] = df_authors_re['Author'].replace(names_update)


In [390]:
add_lines = {'Author': ['Mike Martin',
        'Daniel Zimprich', 
        'Noa Garcia', 
        'Gregory S. Parks', 
        'Rodney M. Miller', 
        'Ashadi Ashadi', 
        'Cecilia O. Alm',
        'Reeta Karra',
        'Jennifer Lyke',
        'Yong Li',
        'Jana Vasickova',
        'Dorota Groffik',
        'Sharon Luise Njet-Moi Tjin A Tsoi',
        'Andries Koster'
        ],
    'Title' : ['Gender differences in level and change in cognitive functioning : Results from the Longitudinal Aging Study',
        'Gender differences in level and change in cognitive functioning : Results from the Longitudinal Aging Study',
        'Gender and Racial Bias in VisualQuestion Answering Datasets',
        'Eighteen Million Cracks: Genders Role in the 2008 Presidential Campaign',
        'If the Shoe Fits: A Historical Exploration of Gender Bias in the U.S. Sneaker Industry',
        "Gender differences in students’ attitudes toward science: An analysis of students’ science process skill using testlet instrument",
        'Analyzing Gender Bias in Student Evaluations Acknowledgments',
        'Gender Bias in Availability of School Education in Villages - A Study of Kalisindh Thermal Power Project',
        'Gender Differences in Perceived Illness, Stress, and Coping in Undergraduates',
        'Racial/ethnic and gender differences in severity of diabetes-related complications, health care resource use, and costs in a Medicare population.',
        'Gender, Age and Body Mass Differences Influencing the Motivation for Physical Activity among Polish Youths',
        'Gender, Age and Body Mass Differences Influencing the Motivation for Physical Activity among Polish Youths',
        'Gender Differences in Motivation and Learning Preferences of Pharmacists in Lifelong Learning',
        'Gender Differences in Motivation and Learning Preferences of Pharmacists in Lifelong Learning'
]}

df_authors_re = df_authors_re.append(DataFrame(add_lines,columns=['Author', 'Title'])).reset_index(drop=True)
df_authors_re.tail()

C:\Users\afedorova\AppData\Local\Temp\ipykernel_1824\1923523578.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_authors_re = df_authors_re.append(DataFrame(add_lines,columns=['Author', 'Title'])).reset_index(drop=True)


,Title,Year,Author,PublicationType,CitationCount,URL
39211,Racial/ethnic and gender differences in severi...,NaN,Yong Li,NaN,NaN,NaN
39212,"Gender, Age and Body Mass Differences Influenc...",NaN,Jana Vasickova,NaN,NaN,NaN
39213,"Gender, Age and Body Mass Differences Influenc...",NaN,Dorota Groffik,NaN,NaN,NaN
39214,Gender Differences in Motivation and Learning ...,NaN,Sharon Luise Njet-Moi Tjin A Tsoi,NaN,NaN,NaN
39215,Gender Differences in Motivation and Learning ...,NaN,Andries Koster,NaN,NaN,NaN


In [722]:
titles_drop2 = ['Gender-biased Diagnosing', 
    'Opiorphin Secretion Pattern in Healthy Volunteers: Gender Difference andOrgan Specificity',
    "Gender Bias in Women's Health, Obstetric, and Neonatal Nursing.",
    'Featured Article: Gender Bias in Pediatric Pain Assessment',
    'Gender Difference in the Effects of COVID-19 Pandemic on Mechanical Reperfusion and 30-Day Mortality for STEMI: Results of the ISACS-STEMI COVID-19 Registry']

i_toDrop = df_authors_re[df_authors_re['Title'].isin(titles_drop2)].index
df_authors_re = df_authors_re.drop(i_toDrop)

In [389]:
print(len(df_authors_re))
i_noSpace = df_authors_re[~df_authors_re.Author.str.contains('\s')].index
df_authors_re = df_authors_re.drop(i_noSpace)
print(len(df_authors_re))

39268
39202


#### Other strange cases

Rename some titles, that don't correspond fully to original paper title.

In [394]:
# 'https://www.semanticscholar.org/paper/b5612cc672817a7d65faf8c273550858b63790a2'
rename_title = {'Re; Pandit SR, Venugopal P, Keshavamurthy R, Chawla A. Challenges and gender-based differences for women in the Indian urological workforce: Results of a survey. Indian J Urol 2022;38:282-6':\
        'Challenges and gender-based differences for women in the Indian urological workforce: Results of a survey',
    'SUCCESSFUL TITLE VII CLAIM FOR IMPLICIT GENDER BIAS ? A . Implicit Bias : A Definition B . Implicit Gender Bias , Title VII , and the Clinton Campaign':\
        'Eighteen Million Cracks: Genders Role in the 2008 Presidential Campaign' }

df_authors_re['Title'] = df_authors_re.Title.replace(rename_title)

name_toDrop = df_authors_re[df_authors_re.Author == 'for the Colon Cancer Family Registry'].index
df_authors_re = df_authors_re.drop(name_toDrop)

Check the rename results

In [396]:
df_authors_re['Title'][df_authors_re['Title'].isin(rename_title.values())]

Series([], Name: Title, dtype: object)

In [397]:
df_authors_re.to_csv('data/authors_2012-2023_regex.csv')

### Find rows with full names

In [17]:
df_authors_re = read_csv('data/df_authors_full.csv')
df_authors_reduced = df_authors_re[df_authors_re.Title.str.contains('[Gg]ender(.*)[Bb]ias|[Gg]ender(.*)[Gg]ap|[Gg]ender(.*)[iI]nequalit')] #.to_csv('data/df_authors_re-genderDifference.csv')

C:\Users\afedorova\AppData\Local\Temp\ipykernel_12032\5840737.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_authors_reduced = df_authors_re[df_authors_re.Title.str.contains('[Gg]ender(.*)[Bb]ias|[Gg]ender(.*)[Gg]ap|[Gg]ender(.*)[iI]nequalit')] #.to_csv('data/df_authors_re-genderDifference.csv')


In [19]:
df_authors_re[~df_authors_re.Title.isin(df_authors_reduced.Title)].to_csv('data/df_authors_re-OnlyGenderDifference.csv')

create two DF, with full names and with short names

In [486]:
df_authors_full = df_authors_re[~df_authors_re.Author.str.contains('^[\S]\.\s[\S].*|^[\S]\.\s[\S]\.|^[\S]\.[\S].')].reset_index(drop=True)
    #~df_authors_re.Author.str.contains('^[\S]\.\s[\S]\.*]')]
df_authors_short = df_authors_re[df_authors_re.Author.str.contains('^[\S]\.\s[\S].*|^[\S]\.\s[\S]\.|^[\S]\.[\S].')].reset_index(drop=True)
    #df_authors_re.Author.str.contains('^[\S]\.\s[\S]\.')]
print('Length of DF with full names is ', len(df_authors_full), '\n', 'Length of DF with short names is ', len(df_authors_short), sep='')

Length of DF with full names is 15516
Length of DF with short names is 23700


In [170]:
df_authors_full['Author'][df_authors_full.FirstName.isna()] #[df_authors_full.FirstName.str.contains('^[\S]\.\s[\S].*|^[\S]\.\s[\S]\.|^[\S]\.[\S].')]

15510        R. Dietz
15511      R. Pearson
15512        M. Semak
15513       C. Willis
15527       A. Little
             ...     
34325     P. Zeppegno
34326    S. Galderisi
34327          M. Maj
34328       S. Mandal
34329        P. Nayak
Name: Author, Length: 8805, dtype: object

check if there are any authors in both df due to a mistake in regex

In [399]:
df_authors_full[df_authors_full['Author'].isin(df_authors_short['Author'])] #[df_authors.Author.str.contains('^[A-Za-z]{1}\.\s[A-Za-z-]{2,50}')]

,Title,Year,Author,PublicationType,CitationCount,URL


check some problematic cases

In [400]:
print(df_authors_short['Author'][df_authors_short['Author']=='G. Ríos-Muñoz'],
    df_authors_short['Author'][df_authors_short['Author']=='A. van Mark'],
    df_authors_short['Author'][df_authors_short['Author']=='Á. M. Fidalgo'],
    df_authors_short['Author'][df_authors_short['Author']=='C. Çöteli'], 
    df_authors_short['Author'][df_authors_short['Author']=='K. Židová'], 
    df_authors_short['Author'][df_authors_short['Author']=='Á. Szabó-Morvai'], sep='\n')

37272    G. Ríos-Muñoz
Name: Author, dtype: object
34619    A. van Mark
Name: Author, dtype: object
20307    Á. M. Fidalgo
Name: Author, dtype: object
27359    C. Çöteli
Name: Author, dtype: object
3357    K. Židová
Name: Author, dtype: object
38231    Á. Szabó-Morvai
Name: Author, dtype: object


Print the info about resulting dataframes

In [89]:
#print(f'In the final search there are {df_authors_re.Title.nunique()} publications from {df_authors_re.Author.nunique()} authors')
print(f'{df_authors_full.Title.nunique()} publications have {df_authors_full.Author.nunique()} full names')
print(f'{df_authors_full[df_authors_full.Title.isin(df_authors_short.Title.unique())].Title.nunique()} publications have both forms of author names')
print(f'{df_authors_short[df_authors_short.Title.isin(df_authors_full.Title.unique())].Author.nunique()} author names have to be corrected')
print(f'{df_authors_full[~df_authors_full.Title.isin(df_authors_short.Title.unique())].Title.nunique()} publications \
have {df_authors_short[~df_authors_short.Title.isin(df_authors_full.Title.unique())].Author.nunique()} authors with only short forms ')

10523 publications have 33714 full names
3326 publications have both forms of author names
8101 author names have to be corrected
7197 publications have 0 authors with only short forms 


In [488]:
df_authors_full.Year.value_counts(sort=False)

2012.0    1190
2013.0    1106
2014.0    1183
2015.0    1283
2016.0    1377
2017.0    1510
2018.0    1506
2019.0    1580
2020.0     633
2021.0    1889
2022.0    1817
2023.0     428
Name: Year, dtype: int64

In [95]:
def name_separator(fullName):
    """Splits a full name into first name, last name and middle name
    
    :param: fullName string with the full name
    :return: firstName string first part of the full name string
    :return: lastName string last part of the full name string
    :return: middleName string if splitted full name contains more than 2 parts, everything in the middle
    """
    name = fullName.split()
    if len(name) > 2:
        firstName = name[0]
        lastName = name[-1]
        middleName = ' '.join(name[1:-1])
        return firstName, lastName, middleName
    elif len(name) == 2:
        firstName = name[0]
        lastName = name[1]
        return firstName, lastName, None

In [ ]:
df_authors_full[['FirstName', 'LastName', 'MiddleName']] = \
    df_authors_full.apply(lambda x: name_separator(x.Author), axis=1, result_type="expand")

In [ ]:
# Alternative: https://stackoverflow.com/questions/38437847/pandas-split-name-column-into-first-and-last-name-if-contains-one-space
# name_df[['first_name','last_name']] = name_df['name'].loc[name_df['name'].str.split().str.len() == 2].str.split(expand=True)

In [490]:
df_authors_full[['Author','FirstName', 'LastName', 'MiddleName']][~df_authors_full['MiddleName'].isna()]

,Author,FirstName,LastName,MiddleName
0,Laura V. Zimmermann,Laura,Zimmermann,V.
1,Seyyed Ahmad Mousavi,Seyyed,Mousavi,Ahmad
4,Margaret S. Stockdale,Margaret,Stockdale,S.
6,Seyyed Ahmad Mousavi,Seyyed,Mousavi,Ahmad
7,Judith Sama Yeba,Judith,Yeba,Sama
...,...,...,...,...
15487,Anna M. Stadelman,Anna,Stadelman,M.
15505,Gregory S. Parks,Gregory,Parks,S.
15506,Rodney M. Miller,Rodney,Miller,M.
15508,Cecilia O. Alm,Cecilia,Alm,O.


In [407]:
import random

test_batch = random.sample(list(df_authors_full['Author'].unique()), 500)
test_batch

['Glenn Hardaker',
 'Gail M. Williams',
 'Roger Kingerlee',
 'Fabian Kosse',
 'Laura N MacMullin',
 'Irina Rets',
 'Emma A Gans',
 'Noah J. Kaplan',
 'Yi-Qing Yang',
 'Danika I. DesRoches',
 'Youngji Kim',
 'Yingping Liu',
 'Lukas Retzer',
 'Emma von Essen',
 'MR Chiappetta',
 "Patricia O'Rourke",
 'Shawn M. Bergman',
 'Elena Manzoni',
 'Shih-Huang Lee',
 'Giulia Antonelli',
 'Maithili S. Umate',
 'Roopa Gurm',
 'Megan Oka',
 'Jessica Greenawalt',
 'Sanjay Singh',
 'Ryuhei So',
 'Ritesh Kumar',
 'Rui Hu',
 'Mengya Yang',
 'Akira Nakayama',
 'Joel R. Anderson',
 'Maria Kanwal',
 'Zeyu Xiao',
 'Ishara Sahama',
 'Sławomir Jandziś',
 'Omar Ruvalcaba',
 'Bas van Leeuwen',
 'Jennifer Gatewood Owens',
 'Corali Bel',
 'Katerina P. Günter',
 'Lalini Ramanathan',
 'Ansh Mittal',
 'Ya-song Wu',
 'Cristiano Ciappei',
 'Christa E. Kiersch',
 'Huda Haider Al Sayed Ahmed',
 'Yoshito  Ikada ',
 'Nik Thompson',
 'Kara Kedrick',
 'Norah Alshareef',
 'Hyunsik Kang',
 'Kagari Shibazaki',
 'Ursula Kessels'

In [408]:
test_batch_info = df_authors_full[['Author', 'FirstName', 'LastName', 'Title', 'URL']].loc[df_authors_full['Author'].isin(test_batch)]
test_batch_info.to_csv('data/test_batch_full_info.csv')

### Group by title to ind out, if all authors have names in the same format

In [75]:
author_gr = df_authors_re.groupby('Title')['Author'].apply(lambda x: "{%s}" % ', '.join(x))
type(author_gr)

pandas.core.series.Series

In [ ]:
#author_gr.str.contains('[A-Za-z]{1}\.\s[A-Za-z]{2,50}')

To update manually:

In [117]:
df_authors_short[df_authors_short.Title.isin(df_authors_full.Title.unique())].to_csv('data/short_names_to_fill.csv')

In [24]:
df_authors_re[df_authors_re.Title == 'Reducing unconscious gender bias through workshop with co-speculative design']

NameError: name 'df_authors_re' is not defined

In [ ]:
7926 'Gender bias and resistance in Edna O’Brien’s “country girls' trilogy”' 'Tsung-Chi Chang'
'Gender Bias Against Foreign Wives : Analysis of International Divorce Cases in Japan' 'Naoko Kato'
'Cultural Linguistic Study on the Chinese Gender-biased Terms against Women' 'Kim Hyun Tae' 'Korean' 'Male'
'Reducing unconscious gender bias through workshop with co-speculative design' 'Hazuki Moyoshi'
'Reducing unconscious gender bias through workshop with co-speculative design' 'Akira Kato'

'Odbal' 'female'

From https://aclanthology.org/2022.ccl-1.46.pdf:
36393-36398


# Is it possible to extract more names from initials?

In [491]:
df_authors_short[['FirstName', 'LastName', 'MiddleName']] = df_authors_short.apply(lambda x: name_separator(x.Author), axis=1, result_type="expand")

In [432]:
df_authors_short[df_authors_short['LastName'].isna()]

,Title,Year,Author,PublicationType,CitationCount,URL,FirstName,LastName,MiddleName


There are some erroneous firstNames, that must be filtered out during search: 
- one letter initials with a dot (eg. 'M.'),
- initials with two letters and dash or dot in between (eg. 'M. C.' or 'M.-C.')
- a title, that was collected as Name (eg. 'Dr.', 'Mr.)

They need to be cleared, but without erasing some names, that have dots at the end (eg. 'Jennifer.')

In [69]:
def update_names_from_RG(paper_title, surname,webdriver='dr', name=None):
    time.sleep(random.randint(1,10))
    query = paper_title.replace(' ', '+') + '+' + surname
    url = f"https://www.researchgate.net/search.Search.html?query={query}&type=publication"
    paper_title = re.sub('[^0-9a-zA-Z ]+', '', paper_title).lower()
    #print('Search for:', paper_title)
    dr.get(url)
    soup = BeautifulSoup(dr.page_source,"lxml")

    results = soup.find_all('div', class_="nova-legacy-v-publication-item__stack nova-legacy-v-publication-item__stack--gutter-m")
        
    for result in results:
        result_title = result.find_all("a", class_="nova-legacy-e-link nova-legacy-e-link--color-inherit nova-legacy-e-link--theme-bare")
        for result_t in result_title:
            result_t = re.sub('[^0-9a-zA-Z ]+', '', result_t.text).lower()
            #print(result_t)
            #print('Found:', re.sub('[^0-9a-zA-Z ]+', '', result_t),         
            if result_t == paper_title:
                results_name = result.find_all('a', class_='nova-legacy-v-person-inline-item')
                for name in results_name:
                    full_name = name.text.split(' ')
                    #print(surname, full_name[-1])
                    if surname.lower() == full_name[-1].lower():
                        if not full_name[0].replace('.', '').isupper():
                            if (full_name[0] not in ['Mr.', 'Dr.', 'Miss., Md.']):
                                first_name = full_name[0]
                                #print(first_name)
                                return first_name

Open Chrome Tab for search

In [71]:
dr = webdriver.Chrome()

Perform search in batches of 50 rows, appending the search result in FirstName column.

Every 500 rows the results are saved as csv.

In [54]:
for i in range(0,11739,50):
    df_authors_short_batch = df_to_expand.iloc[i:i+50]
    df_authors_short_batch['FirstName'] = df_authors_short_batch.apply(lambda x: update_names_from_RG(x.Title, x.LastName, dr), axis=1)
    df_authors_full2 = df_authors_full2.append(df_authors_short_batch, ignore_index=True)
    print(f'{i+50} done')
    if (i + 50) % 500 == 0:
        #dr = webdriver.Chrome()
        df_authors_full2[~df_authors_full.FirstName.isnull()].to_csv('data/authors_full_names_extended.csv')

Results of update and the rate of success (number of update names in proportion to all added names)

In [204]:
print('Added new full names: ',len(df_authors_full[~df_authors_full.FirstName.isnull()]) - 15500, '\n',
    'Could not update names: ', len(df_authors_full[df_authors_full.FirstName.isnull()]), '\n',
    'success rate = ', ((len(df_authors_full[~df_authors_full.FirstName.isnull()]))-15500)/23700, sep='')

Added new full names: 17477
Could not update names: 8805
success rate = 0.7374261603375527


Added new full names: 17477

Could not update names: 8805

success rate = 0.7374261603375527

In [87]:
df_authors_short = df_authors_full[df_authors_full.FirstName.isnull()]

In [164]:
df_authors_full.to_csv('data/df_authors_full.csv')

# Analysis of the short and full names ratio after search in ResearchGate

In [22]:
df_full = df_authors_full[df_authors_full.FirstName.notna()]
df_short = df_authors_full[df_authors_full.FirstName.isna()]
print(len(df_full), len(df_short))

32855 8785


In [23]:
#print(f'In the final search there are {df_authors_re.Title.nunique()} publications from {df_authors_re.Author.nunique()} authors')
print(f'{df_full.Title.nunique()} publications have {df_full.Author.nunique()} full names')
print(f'{df_full[df_full.Title.isin(df_short.Title.unique())].Title.nunique()} publications have both forms of author names')
print(f'{df_short[df_short.Title.isin(df_full.Title.unique())].Author.nunique()} author names have to be corrected')
print(f'{df_short[~df_short.Title.isin(df_full.Title.unique())].Title.nunique()} publications \
have {df_short[~df_short.Title.isin(df_full.Title.unique())].Author.nunique()} authors with only short forms ')

10735 publications have 28733 full names
2577 publications have both forms of author names
6675 author names have to be corrected
735 publications have 1525 authors with only short forms 


Create two dataFrames grouped by Title and the column "Count" shows, how many times the title appears in the initial DataFrame. It shows, how many authors could be identified (for df_full) or could not be identified (df_short) for each title.

In [187]:
df_short_group = df_short.groupby(['Title'])['Title'].count().reset_index(
  name='Count').sort_values(['Count'], ascending=False)
df_full_group = df_full.groupby(['Title'])['Title'].count().reset_index(
  name='Count').sort_values(['Count'], ascending=False)

print_all(df_full_group)

                                                   Title  Count
4235   Gender Gap in Parental Leave Intentions: Evide...     80
10514   Within-job gender pay inequality in 15 countries     29
793    Author Correction: Within-job gender pay inequ...     29
6464   Gender inequality and not female mentors hinde...     28
4779   Gender bias in academia: A lifetime problem th...     22
6635   Gender- and Age-Specific Differences in the As...     22
2798   Gender Difference in the Effects of COVID-19 P...     20
6714   Gender-Specific Differences on the Association...     19
342    Addressing Gender Inequality in Our Discipline...     17
4326   Gender Inequalities in the Dental Workforce: G...     17
9238                   Survey of gender bias in the IPCC     16
4327   Gender Inequalities in the Education of the Se...     16
10085  To be funny or not to be funny: Gender differe...     16
6829   Gender-specific differences in associations of...     15
6467   Gender inequality and restrictive

Have a look, what is a proportion of authors with short and full names for each publication, that needs full names. 

Suggestion: only update manually names, if there is 1 or 2 names missing for a publication. 

We can calculate proportion of short names compared to the number of authrs for each publication and only fill in the short names, if the proportion of missing names is below 60 % (?).

In [ ]:
df_full_group = df_full_group.assign(
    Count_short=lambda df: df_short_group['Count'][df["Title"].isin(df_short_group["Title"])])

df_full_group['Proportion_names'] = df_full_group.apply(lambda x: (x.Count)/(x.Count + x.Count_short), axis=1)
df_full_group['Count_short'] = df_full_group['Count_short'].astype('Int64')
#df_full_group['Proportion_names'] = df_full_group['Proportion_names'].astype('Int64')

df_short_group = df_short_group.assign(
    Count_full=lambda df: df_full_group['Count'][df["Title"].isin(df_full_group["Title"])])
df_short_group['Count_short'] = df_short_group['Count_full'].astype('Int64')
df_short_group['Proportion_names'] = df_full_group.apply(lambda x: (x.Count)/(x.Count + x.Count_full), axis=1)
print_all(df_short_group[['Count','Count_full', 'Proportion_names']].sort_values('Proportion_names', ascending=False))


In [237]:
title_most_popular = [l for l in df_full_group['Title'][df_full_group.Proportion_names > 0.6]]

In [ ]:
df_short[['Title', 'Author']][df_short.Title.isin(title_most_popular)].sort_values('Title')

In [226]:
df_short_name = df_short[df_short.Title.isin(df_full.Title)].groupby(['Author'])['Author'].count().reset_index(
  name='Count').sort_values(['Count'], ascending=False).to_csv('data/most_popular_short_names.csv')
print_all(df_short_name)

None


In [174]:
df_short[df_short.Title.isin(df_full.Title.unique())].to_csv('data/short_names_to_fill.csv')

In [30]:
dict_TitleJournalName['Reconsidering Gender Bias in Intrahousehold Allocation in India']

'The Journal of Development Studies'

In [33]:
df_authors_full['Journal'] = df_authors_full['Title'].map(dict_TitleJournalName)
df_authors_full[['Title', 'Journal']]
print(len(df_authors_full[df_authors_full.Journal.isna()]))

24454


# Determine gender by name
## Genderize.io

In [ ]:
genders = []
for name in [name for name in df_authors_full.FirstName.head()]:
    req = requests.get("https://api.genderize.io?name=" + name) #https://api.genderize.io?name=peter
    result = json.loads(req.text)
    genders.append(result)

print(genders)

## Gender API

In [ ]:
url_genderapi = 'https://gender-api.com/get?name='
key_genderapi = '&key=wTotoGrvGYYVau3ktAD5LBaLDgRWTc8c5728'

genderapi = []

for name in [name for name in df_authors_full.FirstName.head()]:
    req = requests.get(url_genderapi+name+key_genderapi)
    result = json.loads(req.text)
    genderapi.append(result)

print(genderapi)

# In case of emergency

In [8]:
df_authors_full = read_csv('data/df_authors_full.csv')

In [13]:
len(df_authors_full)

41660

In [10]:
df_authors_full.Title = df_authors_full.Title.str.rstrip('.')

In [14]:
df_authors_full[['Title', 'Author', 'FirstName']][df_authors_full.Author == 'M. LaFrance']

,Title,Author,FirstName
27112,Gender Bias in Pediatric Pain Assessment,M. LaFrance,Marianne
28814,Featured Article: Gender Bias in Pediatric Pai...,M. LaFrance,NaN
36011,"""Master"" of none: Institutional language chang...",M. LaFrance,Marianne
36150,"Gendered Anti-Bisexual Bias: Heterosexual, Bis...",M. LaFrance,Marianne


In [15]:
index = df_authors_full[['Title', 'Author', 'FirstName']][df_authors_full.duplicated(['Title', 'Author', 'FirstName'])].index #[df_authors_full.FirstName.isnull()].sort_values('Title')
print(len(index))
df_authors_full = df_authors_full.drop(index)

0


In [16]:
index2 = df_authors_full[df_authors_full.duplicated(['Title', 'Author'], keep=False)][df_authors_full.FirstName.isnull()].index
print(len(index2))
df_authors_full = df_authors_full.drop(index2)

20


C:\Users\afedorova\AppData\Local\Temp\ipykernel_24904\977037083.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  index2 = df_authors_full[df_authors_full.duplicated(['Title', 'Author'], keep=False)][df_authors_full.FirstName.isnull()].index


In [ ]:
df_authors_full = df_authors_full.drop('Unnamed: 0')
df_authors_full.columns()

In [21]:
df_authors_full.columns

Index(['Title', 'Year', 'Author', 'PublicationType', 'CitationCount', 'URL',
       'FirstName', 'LastName', 'MiddleName'],
      dtype='object')

## Manual update

In [2]:
'R. Dietz': 'Richard Dietz',
'R. Pearson':  'Richard Pearson',
'M. Semak': 'Matthew R Semak',
'C. Willis': 'Courtney W Willis',
'D. Syed': 'Deeba N. Syed',
'H. Mukhtar': 'Hasan Mukhtar'

SyntaxError: illegal target for annotation (1957024364.py, line 1)

How is the relative distribution of gender among the researchers, who study gender bias?

How is the temporal dynamics of the female representation as well as the decade forecast?

Check search pattern "women + bias" (as in "bias against women") in df_authors, that are not in df_authors_re

try google scholar with SemanticScholar for author names